In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, StratifiedKFold, train_test_split
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

In [0]:
train_df = pd.read_csv('Training set values.csv',index_col='id')
test_df = pd.read_csv('Test Set values.csv',index_col='id')
target_df = pd.read_csv('Training set Labels.csv',index_col='id')

In [0]:
print("Train Data shape",train_df.shape)
print("Test Data shape",test_df.shape)

Train Data shape (59400, 39)
Test Data shape (14850, 39)


In [0]:
train_df.dtypes

amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_type              object
water_qual

In [0]:
def feature_info(features):
  print('Unique values in the feature:')
  print(features.nunique())
  print('No of Null values:')
  print(features.isna().sum())

In [0]:
feature_info(train_df)

Unique values in the feature:
amount_tsh                  98
date_recorded              356
funder                    1897
gps_height                2428
installer                 2145
longitude                57516
latitude                 57517
wpt_name                 37400
num_private                 65
basin                        9
subvillage               19287
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
recorded_by                  1
scheme_management           12
scheme_name               2696
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_g

In [0]:
def convert_datetime(df, col):
  df[col] = pd.to_datetime(df[col])
  df['year'] = df[col].dt.year
  df['month'] = df[col].dt.month 
  df['day'] = df[col].dt.day 
  
  return None

In [0]:
convert_datetime(train_df,'date_recorded')
convert_datetime(test_df,'date_recorded')

In [0]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   amount_tsh             59400 non-null  float64       
 1   date_recorded          59400 non-null  datetime64[ns]
 2   funder                 55765 non-null  object        
 3   gps_height             59400 non-null  int64         
 4   installer              55745 non-null  object        
 5   longitude              59400 non-null  float64       
 6   latitude               59400 non-null  float64       
 7   wpt_name               59400 non-null  object        
 8   num_private            59400 non-null  int64         
 9   basin                  59400 non-null  object        
 10  subvillage             59029 non-null  object        
 11  region                 59400 non-null  object        
 12  region_code            59400 non-null  int64         
 1

For everytime a longitude is recorded 0 the corresponding latitude is marked as -2.000000e-08, which is actually absurd because these are garbage values and cant be the correct ones. We need to replace that with the suitable one. Similarly for every numerical column where there is 0  we can think that its equivalent to missing and compute the suitable imputations to it 

In [0]:
train_copy =  train_df.copy()
test_copy = test_df.copy()
def clean_num(df):
 cols = ['amount_tsh', 'gps_height', 'longitude', 'latitude','population','construction_year']
 df[cols] = df[cols].replace({0:np.nan, -2.000000e-08:np.nan})

 for feature in cols:
   df[feature] = df[feature].fillna(df.groupby('region')[feature].transform('mean'))
 
 for feature in cols:
   med =  df[feature].median()
   df[feature] = df[feature].fillna(med)

 return df

In [0]:
train_copy=clean_num(train_copy)
test_copy=clean_num(test_copy)

In [0]:
train_copy.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year,month,day
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.000000,2011-03-14,Roman,1390.000000,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109.0,True,GeoData Consultants Ltd,VWC,Roman,False,1999.0,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,2011,3,14
8776,584.158038,2013-03-06,Grumeti,1399.000000,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280.0,NaN,GeoData Consultants Ltd,Other,NaN,True,2010.0,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,3,6
34310,25.000000,2013-02-25,Lottery Club,686.000000,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250.0,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009.0,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,2013,2,25
67743,148.481268,2013-01-28,Unicef,263.000000,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58.0,True,GeoData Consultants Ltd,VWC,NaN,True,1986.0,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,2013,1,28
19728,725.095986,2011-07-13,Action In A,1199.643052,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,200.0,True,GeoData Consultants Ltd,NaN,NaN,True,2000.0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2011,7,13


Data Cleaning for the categorical columns 

In [0]:
num_cols = ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private','population','construction_year']
cat_cols = ['funder','installer','wpt_name','basin', 'subvillage', 'region', 'lga', 'ward','public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit','extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']

In [0]:
def clean_cat(df):
  df[cat_cols] = df[cat_cols].replace({'0':np.nan, 'not known':np.nan, 'Not Known':np.nan, 'None':np.nan, 'unknown':np.nan, 'none':np.nan, '-':np.nan, '##':np.nan, 'not kno':np.nan, 'Unknown Installer':np.nan})  
  for features in cat_cols: 
    df[cat_cols]=df[cat_cols].fillna(df.mode().iloc[0])
  return df


In [0]:
train_copy = clean_cat(train_copy)
test_copy = clean_cat(test_copy)

In [0]:
train_copy.head(10)

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year,month,day
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.000000,2011-03-14,Roman,1390.000000,Roman,34.938093,-9.856322,Shuleni,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109.000000,True,GeoData Consultants Ltd,VWC,Roman,False,1999.000000,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,2011,3,14
8776,584.158038,2013-03-06,Grumeti,1399.000000,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280.000000,True,GeoData Consultants Ltd,Other,K,True,2010.000000,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,3,6
34310,25.000000,2013-02-25,Lottery Club,686.000000,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250.000000,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009.000000,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,2013,2,25
67743,148.481268,2013-01-28,Unicef,263.000000,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58.000000,True,GeoData Consultants Ltd,VWC,K,True,1986.000000,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,2013,1,28
19728,725.095986,2011-07-13,Action In A,1199.643052,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,200.000000,True,GeoData Consultants Ltd,VWC,K,True,2000.000000,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2011,7,13
9944,20.000000,2011-03-13,Mkinga Distric Coun,667.914800,DWE,39.172796,-4.765587,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1.000000,True,GeoData Consultants Ltd,VWC,Zingibali,True,2009.000000,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,groundwater,communal standpipe multiple,communal standpipe,2011,3,13
19816,3746.666667,2012-10-01,Dwsp,1350.981707,DWSP,33.362410,-3.766365,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,428.359756,True,GeoData Consultants Ltd,VWC,K,True,2002.621951,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,2012,10,1
54551,3746.666667,2012-10-09,Rwssp,1350.981707,DWE,32.620617,-4.226198,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,428.359756,True,GeoData Consultants Ltd,VWC,K,True,2002.621951,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,2012,10,9
53934,725.095986,2012-11-03,Wateraid,1199.643052,Water Aid,32.711100,-5.146712,Kwa Ramadhan Musa,0,Lake Tanganyika,Imalauduki,Tabora,14,6,Tabora Urban,Itetemia,200.000000,True,GeoData Consultants Ltd,VWC,K,True,2000.000000,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump,2012,11,3


In [0]:
def dropping_off (df):
  drop_cols = ['longitude','latitude','num_private','ward','recorded_by',
              'extraction_type_group','extraction_type_class',
              'region_code','waterpoint_type_group','source_type',
             'payment_type','quality_group','quantity_group','date_recorded']
  df= df.drop(drop_cols,axis=1)
  return df

In [0]:
train_copy = dropping_off(train_copy)
test_copy =  dropping_off(test_copy)  

In [0]:
  #numer_feat = ['amount_tsh','gps_height','num_private','population','construction_year']
  #scaler =  MinMaxScaler()
  #train_copy[numer_feat] = scaler.fit_transform(train_copy[numer_feat])
  #test_copy[numer_feat]  = scaler.transform(test_copy[numer_feat])

In [0]:
print("Training shape before dummifying",train_copy.shape)
print("Test set shape before dummifying",test_copy.shape)

Training shape before dummifying (59400, 28)
Test set shape before dummifying (14850, 28)


In [0]:
feature_info(train_copy)
feature_info(test_copy)

Unique values in the feature:
amount_tsh             114
funder                1894
gps_height            2435
installer             2140
wpt_name             37397
basin                    9
subvillage           19286
region                  21
district_code           20
lga                    125
population            1057
public_meeting           2
scheme_management       11
scheme_name           2693
permit                   2
construction_year       71
extraction_type         18
management              11
management_group         4
payment                  6
water_quality            7
quantity                 4
source                   9
source_class             2
waterpoint_type          7
year                     5
month                   12
day                     31
dtype: int64
No of Null values:
amount_tsh           0
funder               0
gps_height           0
installer            0
wpt_name             0
basin                0
subvillage           0
region               

In [0]:
y_train = target_df['status_group']

In [0]:
cat_cols_updated = ['funder','installer','wpt_name','basin', 'subvillage', 'region', 'lga','public_meeting','scheme_management', 'scheme_name', 'permit','extraction_type', 'management', 'management_group', 'payment', 'water_quality','quantity', 'source','source_class', 'waterpoint_type']
for col in cat_cols_updated:
  print (col, train_copy[col].nunique())
  print (col, test_copy[col].nunique())

funder 1894
funder 977
installer 2140
installer 1089
wpt_name 37397
wpt_name 10838
basin 9
basin 9
subvillage 19286
subvillage 8443
region 21
region 21
lga 125
lga 125
public_meeting 2
public_meeting 2
scheme_management 11
scheme_management 11
scheme_name 2693
scheme_name 1786
permit 2
permit 2
extraction_type 18
extraction_type 17
management 11
management 11
management_group 4
management_group 4
payment 6
payment 6
water_quality 7
water_quality 7
quantity 4
quantity 4
source 9
source 9
source_class 2
source_class 2
waterpoint_type 7
waterpoint_type 7


'funder','installer','subvillage','wpt_name','scheme_name','lga' these are the columns which have high cardinality so we can then transform least frequent columns into 'other' category. 


Transforming the column 'funder'

In [0]:
train_copy.funder.value_counts().head(10)

Government Of Tanzania    13525
Danida                     3114
Hesawa                     2202
Rwssp                      1374
World Bank                 1349
Kkkt                       1287
World Vision               1246
Unicef                     1057
Tasaf                       877
District Council            843
Name: funder, dtype: int64

In [0]:
def funder_cl(row):  
    if row['funder']=='Government Of Tanzania':
        return 'gov'
    elif row['funder']=='Danida':
        return 'danida'
    elif row['funder']=='Hesawa':
        return 'hesawa'
    elif row['funder']=='Rwssp':
        return 'rwssp'
    elif row['funder']=='World Bank':
        return 'world_bank'    
    elif row['funder']=='Kkkt':
        return 'Kkkt'
    elif row['funder']=='World Vision':
        return 'World Vision'
    elif row['funder']=='Unicef':
        return 'Unicef'
    elif row['funder']=='Tasaf':
        return 'Tasaf'
    elif row['funder']=='District Council':
        return 'District Council'
    else:
        return 'other'
    
train_copy['funder'] = train_copy.apply(lambda row: funder_cl(row), axis=1)
test_copy['funder'] = test_copy.apply(lambda row: funder_cl(row), axis=1)

transforming the column installer

In [0]:
train_copy.installer.value_counts().head(10)

DWE                   21863
Government             1825
RWE                    1206
Commu                  1060
DANIDA                 1050
KKKT                    898
Hesawa                  840
TCRS                    707
Central government      622
CES                     610
Name: installer, dtype: int64

In [0]:
def installer_cl(row):  
    if row['installer']=='DWE':
        return 'dwe'
    elif row['installer']=='Government':
        return 'gov'
    elif row['installer']=='RWE':
        return 'rwe'
    elif row['installer']=='Commu':
        return 'commu'
    elif row['installer']=='DANIDA':
        return 'danida'    
    elif row['installer']=='Kkkt':
        return 'Kkkt'
    elif row['installer']=='Hesawa':
        return 'hesawa'
    elif row['installer']=='TCRS':
        return 'tcrs'
    elif row['installer']=='Central Government':
        return 'central gov'
    elif row['installer']=='CES':
        return 'ces'
    else:
        return 'other'
    
train_copy['installer'] = train_copy.apply(lambda row: installer_cl(row), axis=1)
test_copy['installer'] = test_copy.apply(lambda row: installer_cl(row), axis=1)

Transforming wpt_name column

In [0]:
train_copy.wpt_name.value_counts().head(10)

Shuleni            5333
Zahanati            830
Msikitini           535
Kanisani            323
Bombani             271
Sokoni              260
Ofisini             254
School              208
Shule Ya Msingi     199
Shule               152
Name: wpt_name, dtype: int64

In [0]:
def wpt_name_cl(row):  
    if row['wpt_name']=='Shuleni':
        return 'shuleni'
    elif row['wpt_name']=='Zahanati':
        return 'zahanati'
    elif row['wpt_name']=='Msikitini':
        return 'msikitini'
    elif row['wpt_name']=='Kanisani':
        return 'kanisani'
    elif row['wpt_name']=='Bombani':
        return 'bombani'    
    elif row['wpt_name']=='Sokoni':
        return 'sokoni'
    elif row['wpt_name']=='Ofisini':
        return 'Ofisini'
    elif row['wpt_name']=='School':
        return 'school'
    elif row['wpt_name']=='Shule Ya Msingi':
        return 'shule ya msingi'
    elif row['wpt_name']=='shule':
        return 'shule'
    else:
        return 'other'
    
train_copy['wpt_name'] = train_copy.apply(lambda row: wpt_name_cl(row), axis=1)
test_copy['wpt_name'] = test_copy.apply(lambda row: wpt_name_cl(row), axis=1)

Transforming Sub-Village

In [0]:
train_copy.subvillage.value_counts().head(10)

Madukani    880
Shuleni     506
Majengo     502
Kati        373
Mtakuja     262
Sokoni      232
M           187
Muungano    172
Mbuyuni     164
Mlimani     152
Name: subvillage, dtype: int64

In [0]:
def subvillage_cl(row):  
    if row['subvillage']=='Madukani':
        return 'madukani'
    elif row['subvillage']=='Shuleni':
        return 'shuleni'
    elif row['subvillage']=='Majengo':
        return 'majengo'
    elif row['subvillage']=='Kati':
        return 'kati'
    elif row['subvillage']=='Mtakuja':
        return 'Mtakuja'    
    elif row['subvillage']=='Sokoni':
        return 'sokoni'
    elif row['subvillage']=='M':
        return 'm'
    elif row['subvillage']=='Muungano':
        return 'muungano'
    elif row['subvillage']=='Mbuyuni':
        return 'mbuyuni'
    elif row['subvillage']=='Mlimani':
        return 'mlimani'
    else:
        return 'other'
    
train_copy['subvillage'] = train_copy.apply(lambda row: subvillage_cl(row), axis=1)
test_copy['subvillage'] = test_copy.apply(lambda row: subvillage_cl(row), axis=1)

Transforming last column

In [0]:
train_copy.scheme_name.value_counts().head(10)

K                              29525
Borehole                         546
Chalinze wate                    405
M                                400
DANIDA                           379
Government                       320
Ngana water supplied scheme      270
wanging'ombe water supply s      261
wanging'ombe supply scheme       234
I                                229
Name: scheme_name, dtype: int64

In [0]:
def scheme_name_cl(row):  
    if row['scheme_name']=='K':
        return 'k'
    elif row['scheme_name']=='Borehole':
        return 'borehole'
    elif row['scheme_name']=='Chalinze wate':
        return 'chalinze wate'
    elif row['scheme_name']=='M':
        return 'm'
    elif row['scheme_name']=='DANIDA':
        return 'danida'    
    elif row['scheme_name']=='Government':
        return 'gov'
    elif row['scheme_name']=='Ngana water supplied scheme':
        return 'ngana'
    elif row['scheme_name']=="wanging'ombe water supply s":
        return "wanging'ombe water supply s"
    elif row['scheme_name']=="wanging'ombe supply scheme":
        return "wanging'ombe supply scheme"
    elif row['scheme_name']=='I':
        return 'i'
    else:
        return 'other'
    
train_copy['scheme_name'] = train_copy.apply(lambda row: scheme_name_cl(row), axis=1)
test_copy['scheme_name'] = test_copy.apply(lambda row: scheme_name_cl(row), axis=1)

Transforming the 'lga' column

In [0]:
train_copy.lga.value_counts().head(10)

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
Kilosa          1094
Kasulu          1047
Mbozi           1034
Meru            1009
Bagamoyo         997
Name: lga, dtype: int64

In [0]:
def lga_cl(row):  
    if row['lga']=='Njombe':
        return 'njombe'
    elif row['lga']=='Arusha Rural':
        return 'arusha'
    elif row['lga']=='Moshi Rural':
        return 'moshi'
    elif row['lga']=='Bariadi':
        return 'bariadi'
    elif row['lga']=='Rungwe':
        return 'rungwe'    
    elif row['lga']=='Kilosa':
        return 'kilosa'
    elif row['lga']=='Kasulu':
        return 'kasulu'
    elif row['lga']=="Mbozi":
        return "mbozi"
    elif row['lga']=="Meru":
        return "meru"
    elif row['lga']=='Bagamoyo':
        return 'bagamoyo'
    else:
        return 'other'
    
train_copy['lga'] = train_copy.apply(lambda row: lga_cl(row), axis=1)
test_copy['lga'] = test_copy.apply(lambda row: lga_cl(row), axis=1)

In [0]:
print("The shape of training set after dropping high cardinality columns",train_copy.shape)
print("The shape of test set after dropping high cardinality columns",test_copy.shape)

The shape of training set after dropping high cardinality columns (59400, 28)
The shape of test set after dropping high cardinality columns (14850, 28)


In [0]:
train_copy.columns

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'district_code', 'lga', 'population',
       'public_meeting', 'scheme_management', 'scheme_name', 'permit',
       'construction_year', 'extraction_type', 'management',
       'management_group', 'payment', 'water_quality', 'quantity', 'source',
       'source_class', 'waterpoint_type', 'year', 'month', 'day'],
      dtype='object')

In [0]:
categorial_cols = ['funder','installer','wpt_name','subvillage','basin', 'region','lga', 'public_meeting',
       'scheme_management', 'permit', 'extraction_type','scheme_name',
       'management', 'management_group', 'payment', 'water_quality',
       'quantity', 'source', 'source_class', 'waterpoint_type']
def dummies(data):
  for cc in categorial_cols:
    dummies = pd.get_dummies(data[cc])
    dummies = dummies.add_prefix("{}#".format(cc))
    data.drop(cc, axis=1, inplace=True)
    data = data.join(dummies)
  return data

In [0]:
train_copy = dummies(train_copy)
test_copy = dummies(test_copy)

In [0]:
print("The shape of training set after dropping high cardinality columns",train_copy.shape)
print("The shape of test set after dropping high cardinality columns", test_copy.shape)

The shape of training set after dropping high cardinality columns (59400, 186)
The shape of test set after dropping high cardinality columns (14850, 185)


There is a mismatch between the number of columns so I will go ahead and remove the columns which arent there in both.

In [0]:
x = list(train_copy.columns)
y = list(test_copy.columns)

In [0]:
def mismatch(x,y):
  tr=[]
  te=[]
  for i in range(0,len(x)) :
    if x[i] not in y:
      tr.append(x[i])
  for j in range(0,len(y)) :
    if y[j] not in x:
      te.append(y[j])
  return tr,te

list_tr,list_te = mismatch(x,y)

In [0]:
train_copy =  train_copy.drop(list_tr,axis=1)
test_copy = test_copy.drop(list_te,axis=1)

In [0]:
c = list(train_copy.columns.values)
d = list(test_copy.columns.values)

c == d

True

In [0]:
print("The shape of training set ",train_copy.shape)
print("The shape of test set ", test_copy.shape)

The shape of training set  (59400, 185)
The shape of test set  (14850, 185)


In [0]:
train_copy.to_csv("training.csv",index=True)
test_copy.to_csv("test.csv",index=True)